<img align="left" src="https://www.taborelec.com/Pictures/logo.png">

# Test SCPI Command ':XINStrument:SYNChronize:FOLLowers'

## Test Setup & Purpose

The purpose of the test is to check the functionality of the SCPI command:
 :XINStrument:SYNChronize:FOLLowers
 
The command enables to synchronize the clocks of several instruments in the same chassis.

Test Setup:
* The Chasis should inlcude two Pxx84M device. 
* There Should be a connection wire between refClk-out of instrument 1 to refClk-in of instrument 2
(see attached image)
* The test should run by connecting to a server - please run the following : TEPAdminTestServer.exe
  select the options: 1, 2 and 5 


### Before we start

This example is designed to work on a LAN based communication-service (usually on benchtop/desktop)<br>
In this case, when there's a LAN communication, this example uses the class `TEVisaInst` from `tevisainst`.<br>


In [97]:
%matplotlib notebook
import numpy as np
import time
import ipywidgets as widgets
import matplotlib.pyplot as plt

In [98]:
import os
import gc
import sys
srcpath = os.path.realpath('../SourceFiles')
sys.path.append(srcpath)
from teproteus import TEProteusAdmin, TEProteusInst
from tevisainst import TEVisaInst

In [99]:
# Connect to instrument

inst = None
admin = None

conn_msg_str = ''

def disconnect():
    global inst
    global admin
    if inst is not None:
        try:
            inst.close_instrument()            
        except:
            pass
        inst = None
    if admin is not None:
        try:
            admin.close_inst_admin()
        except:
            pass
        admin = None
    gc.collect()

def connect_to_pxi_slot(slot_id):
    global inst
    global admin
    try:
        disconnect()        
        admin = TEProteusAdmin()
        admin.open_inst_admin()        
        inst = admin.open_instrument(slot_id, reset_hot_flag=True)
    except:
        pass

def connect_to_lan_server(ip_address):
    global inst
    try:
        disconnect()
        inst = TEVisaInst(ip_address, port=5025)
    except:
        pass

def connect(ipaddr_or_slotid):
    try:
        disconnect()
        if isinstance(ipaddr_or_slotid, str) and '.' in ipaddr_or_slotid:
            connect_to_lan_server(ipaddr_or_slotid)
        else:
            connect_to_pxi_slot(ipaddr_or_slotid)
    except:
        pass

addr_textbox = widgets.Text(
    value='',
    placeholder='IP Address or Slot Id',
    description='Address:',
    disabled=False
)

output = widgets.Output()

def print_conn_message(msg_str):
    global output
    global conn_msg_str
    with output:
        if len(conn_msg_str) > 0:
            print('\r', end='')
            print(' ' * len(conn_msg_str), end='')
            print('\r', end='')
        conn_msg_str = str(msg_str)
        if len(conn_msg_str) > 0:
            print(conn_msg_str, end='')

def on_button_connect_clicked(b):
    global inst
    global addr_textbox
    print_conn_message('')
    disconnect()
    if addr_textbox is not None:
        conn_str = addr_textbox.value
        if conn_str:
            print_conn_message('Connecting to {0} ..'.format(conn_str))
            connect(conn_str)
            print_conn_message('')
    if inst is not None:
        idn_str = inst.send_scpi_query('*IDN?')
        if idn_str:
            print_conn_message('Connected to: ' + idn_str.strip())
    
def on_button_disconnect_clicked(b):
    print_conn_message('')
    disconnect()

btn_connect = widgets.Button(description="Connect")
btn_disconnect = widgets.Button(description="Disconnect")

btn_connect.on_click(on_button_connect_clicked)
btn_disconnect.on_click(on_button_disconnect_clicked)

display(addr_textbox, btn_connect, btn_disconnect, output)

Text(value='', description='Address:', placeholder='IP Address or Slot Id')

Button(description='Connect', style=ButtonStyle())

Button(description='Disconnect', style=ButtonStyle())

Output()

In [101]:
# Set the default paranoia-level (0, 1 or 2)
inst.default_paranoia_level = 2 # good for debugging

# Clear error-list and reset the instrument
inst.send_scpi_cmd('*CLS; *RST')

AttributeError: 'NoneType' object has no attribute 'default_paranoia_level'

In [100]:
# Get the model name
model = inst.send_scpi_query(":SYST:iNF:MODel?")
print("Model: " + model)

# Infer the natural DAC waveform format
if 'P908' in model:
    dac_mode = 8
else:
    dac_mode = 16
    


print("DAC waveform format: {0} bits-per-point".format(dac_mode))

max_dac = 2 ** dac_mode - 1
half_dac = max_dac / 2.0
print('Max DAC wave-point level: {0}'.format(max_dac))
print('Half DAC wave-point level: {0}'.format(half_dac))

# Get number of channels
resp = inst.send_scpi_query(":INST:CHAN? MAX")
print("Number of channels: " + resp)
num_channels = int(resp)

# Get the maximal number of segments
resp = inst.send_scpi_query(":TRACe:SELect:SEGMent? MAX")
print("Max segment number: " + resp)
max_seg_number = int(resp)

# Get the available memory in bytes of wavform-data (per DDR):
resp = inst.send_scpi_query(":TRACe:FREE?")
arbmem_capacity = int(resp)
print("Available memory per DDR: {0:,} wave-bytes".format(arbmem_capacity))


AttributeError: 'NoneType' object has no attribute 'send_scpi_query'

## Setup - Writing waveform data to segment

The command `:TRACE:DATA [<offset-in-bytes>] #<binary-header><binary-data>`  
writes the given block of binary data to the specified offset in the selected segment of the selected channel.

#### The header of the binary-data block
The SCPI protocol uses text-based messages, and the new-line character denotes end of a SCPI statement.<br>
In case of binary-data the sender must send a header that defines the size of the binary block before the data  
because the new-line character can be a part of the data.<br>

The header is composed of `'#'` character followed by a sequence of between 2 and 10 decimal digits, where the<br>
first digits denotes how many digits follows it (between 1 and 9) and the rest of the digits denote the size of the<br>
binary-block in bytes. For example, `#3256` denotes block of 256 bytes, and `#41024` denotes block of 1024 bytes.

Download waves to several segments and play different segment in each channel.  


In [87]:
# Build 12 waveforms

seglen = 4096
ncycles = 2

waves = [ None for _ in range(num_channels)]

#Create Square wave
    
for i in range(num_channels):
    cyclelen = seglen / ncycles

    x = np.linspace(start=0, stop=seglen, num=seglen, endpoint=False)
    y = np.fmod(x, cyclelen)
    y = (y <= cyclelen / 2) * max_dac
    y = np.round(y)
    y = np.clip(y, 0, max_dac)
    if dac_mode == 16:
        waves[i] = y.astype(np.uint16)
    else:
        waves[i] = y.astype(np.uint8)
 

In [88]:
def DownloadSeg(num_channels):
# Download segments
 for ii in range(num_channels):
    ichan = ii
    channb = ichan + 1
    segnum = ii +1
    wav = waves[0]
    print('Download wave to segment {0} of channel {1}'.format(segnum, channb))
    
    # Select channel
    cmd = ':INST:CHAN {0}'.format(channb)
    inst.send_scpi_cmd(cmd)
    
    # Define segment
    cmd = ':TRAC:DEF {0}, {1}'.format(segnum, seglen)
    inst.send_scpi_cmd(cmd)
    
    # Select the segment
    cmd = ':TRAC:SEL {0}'.format(segnum)
    inst.send_scpi_cmd(cmd)
        
    # Send the binary-data:
    inst.write_binary_data(':TRAC:DATA', wav)
        
    resp = inst.send_scpi_query(':SYST:ERR?')
    resp = resp.rstrip()
    if not resp.startswith('0'):
        print('ERROR: "{0}" after writing binary values'.format(resp))

    # Play the specified segment at the selected channel:
    cmd = ':SOUR:FUNC:MODE:SEGM {0}'.format(segnum)
    inst.send_scpi_cmd(cmd)

    # Turn on the output of the selected channel:
    inst.send_scpi_cmd(':OUTP ON')
    
    resp = inst.send_scpi_query(':SYST:ERR?')
    print(resp)

 print()

 resp = inst.send_scpi_query(':SYST:ERR?') 
 print(resp)

#del waves
#del wav

## Test 1 - Download wave & Check Default Synchronized Waveforms
The signals from each instrument should be sychronized. Verify on Scope signals are synchronized.

In [89]:
cmd = ':INST:ACT:SEL 1'
inst.send_scpi_cmd(cmd)
cmd = ':INST:ACT:SEL?'
resp=inst.send_scpi_query(cmd)
print(resp)
DownloadSeg(num_channels)




1
Download wave to segment 1 of channel 1
0, no error
Download wave to segment 2 of channel 2
0, no error
Download wave to segment 3 of channel 3
0, no error
Download wave to segment 4 of channel 4
0, no error

0, no error


In [90]:
cmd = ':INST:ACT:SEL 2'
inst.send_scpi_cmd(cmd)
cmd = ':INST:ACT:SEL?'
resp=inst.send_scpi_query(cmd)
print(resp)
DownloadSeg(num_channels)


2
Download wave to segment 1 of channel 1
0, no error
Download wave to segment 2 of channel 2
0, no error
Download wave to segment 3 of channel 3
0, no error
Download wave to segment 4 of channel 4
0, no error

0, no error


### Test 2
The system should is configured by default to synchronized mode. So the following expected results are shown in the comments line of each box


In [91]:

cmd = ':INST:ACT:SEL 1'

inst.send_scpi_cmd(cmd)
cmd = ':XINS:SYNC:FOLL?'
resp=inst.send_scpi_query(cmd)
print(resp)

#Expected result: 1


1


In [92]:
cmd = ':INST:ACT:SEL 2'

inst.send_scpi_cmd(cmd)
cmd = ':XINS:SYNC:FOLL?'
resp=inst.send_scpi_query(cmd)
print(resp)

#Expected result: 0

0


In [93]:
cmd=":FREQ:RAST 2.0e9"
inst.send_scpi_cmd(cmd)
resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)

0

In [94]:
cmd = ':INST:ACT:SEL 2'
inst.send_scpi_cmd(cmd)

cmd = ':XINS:SYNC:ROLE?'
resp=inst.send_scpi_query(cmd)
print(resp)

resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)


#Expected result: FOLL

0
0, no error
FOLL
0, no error


In [ ]:
cmd = ':INST:ACT:SEL 1'
inst.send_scpi_cmd(cmd)

cmd = ':XINS:SYNC:ROLE?'
resp=inst.send_scpi_query(cmd)
print(resp)

resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)

#Expected result: LEAD

## Test 3 - Check Chain Delete
Verify on Scope signals are not synchronized.

In [96]:
cmd = ':INST:ACT:SEL 1'
inst.send_scpi_cmd(cmd)
cmd = ':XINS:SYNC:FOLL 0'
inst.send_scpi_cmd(cmd)

0

In [74]:

cmd = ':XINS:CHAIN:ROLE?'
resp1=inst.send_scpi_query(cmd)
print(resp1)
resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)


VisaIOError: VI_ERROR_CONN_LOST (-1073807194): The connection for the given session has been lost.

In [98]:
cmd = ':XINS:CHAIN:POS?'
resp2=inst.send_scpi_query(cmd)
print(resp2)
resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)


0, no error
2


In [99]:
# Change the segment that being played in channel 1,
# and play all segments one after the other with some time-delay

for ii in range(num_channels):  
    channb = ii + 1
    iseg = ii + 1
    cmd = ':INST:CHAN {0}'.format(channb)
    inst.send_scpi_cmd(cmd)
    cmd = ':SOUR:FUNC:MODE:SEGM {0}'.format(iseg)
    inst.send_scpi_cmd(cmd)
    resp = inst.send_scpi_query(':SYST:ERR?')
    print(resp)
 #   time.sleep(2.5)   

1
1
1
1



## Skew Delay 3 ns between Channels 1 and 3

In [9]:
inst.send_scpi_cmd(':INST:CHAN 1')

inst.send_scpi_cmd(':INST:COUP:SKEW 3e-09')
resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)

0, no error



## Skew Delay -3 ns between Channels 1 and 3

In [10]:
inst.send_scpi_cmd(':INST:COUP:SKEW -3e-09')
resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)

0, no error



## Point Delay between Channels 1 and 2

In [74]:
inst.send_scpi_cmd(':INST:CHAN 1')
inst.send_scpi_cmd(':INST:CHAN:OFFS 10')

resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)

0, no error


In [14]:
# Disconnect
print_conn_message('')
disconnect()